<a href="https://colab.research.google.com/github/ammrit2312/KeyPhrase_BERT/blob/main/KeyPhraseBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Selecting candidate phrases


In [ ]:
import spacy 

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [ ]:
pattern1 = [{'POS':'ADJ', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'*'}, {'POS':'NOUN', 'OP':'+'},{'IS_PUNCT': True, 'OP':'*'},{'POS':'ADP'},{'OP':'?'},{'POS':'ADJ'},{'IS_PUNCT': True, 'OP':'*'},{'POS':'NOUN'}]
#pat = [({'POS':'ADJ', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'*'}, {'POS':'NOUN', 'OP':'+'},{'IS_PUNCT': True, 'OP':'*'},{'POS':'ADP'})?]
pattern2 = [{'POS':'ADJ'},{'IS_PUNCT': True, 'OP':'*'},{'POS':'NOUN'}]
pattern3 = [{'ENT_TYPE':'ORG', 'OP':'+'}]
pattern4 = [{'ENT_TYPE':'GPE', 'OP':'+'}]


matcher.add('Noun', None, pattern1, pattern2, pattern3, pattern4)

In [ ]:
doc = nlp(u'''Cricket is a bat-and-ball game played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. The batting side scores runs by striking the ball bowled at the wicket with the bat (and running between the wickets), while the bowling and fielding side tries to prevent this (by preventing the ball from leaving the field, and getting the ball to either wicket) and dismiss each batter (so they are "out"). Means of dismissal include being bowled, when the ball hits the stumps and dislodges the bails, and by the fielding side either catching the ball after it is hit by the bat, but before it hits the ground, or hitting a wicket with the ball before a batter can cross the crease in front of the wicket. When ten batters have been dismissed, the innings ends and the teams swap roles. The game is adjudicated by two umpires, aided by a third umpire and match referee in international matches. They communicate with two off-field scorers who record the match's statistical information.

Forms of cricket range from Twenty20, with each team batting for a single innings of 20 overs, to Test matches played over five days. Traditionally cricketers play in all-white kit, but in limited overs cricket they wear club or team colours. In addition to the basic kit, some players wear protective gear to prevent injury caused by the ball, which is a hard, solid spheroid made of compressed leather with a slightly raised sewn seam enclosing a cork core layered with tightly wound string.

The earliest reference to cricket is in South East England in the mid-16th century. It spread globally with the expansion of the British Empire, with the first international matches in the second half of the 19th century. The game's governing body is the International Cricket Council (ICC), which has over 100 members, twelve of which are full members who play Test matches. The game's rules, the Laws of Cricket, are maintained by Marylebone Cricket Club (MCC) in London. The sport is followed primarily in the Indian subcontinent, Australasia, the United Kingdom, southern Africa and the West Indies.[1] Women's cricket, which is organised and played separately, has also achieved international standard. The most successful side playing international cricket is Australia, which has won seven One Day International trophies, including five World Cups, more than any other country and has been the top-rated Test side more than any other country.''')


In [ ]:
found_matches = matcher(doc)
phrases = set()
for matcher_id, start, end in found_matches:
  if((start!=end) and (doc[start:end] not in nlp.Defaults.stop_words)):
    phrases.add(str(doc[start:end]))

phrases=list(phrases)
phrases

['the United',
 'the',
 'the International Cricket Council (',
 '(',
 'basic kit',
 'the International',
 'international matches in the second half',
 '19th century',
 'Empire',
 'Cricket Council',
 'Cricket Council (',
 'Council (ICC',
 'Marylebone Cricket',
 'half of the 19th century',
 'Council (',
 'Club',
 'British',
 'Indian subcontinent',
 'Council',
 'London',
 'single innings',
 'the International Cricket',
 'other country',
 'United',
 'the International Cricket Council',
 'United Kingdom',
 'full members',
 'first international matches in the second half',
 'the British Empire',
 'British Empire',
 'matches in the second half',
 'second half',
 'Cricket Club',
 'Kingdom',
 'ICC',
 'statistical information',
 'International',
 'International Cricket Council (',
 'second half of the 19th century',
 'Australasia',
 'the British',
 'third umpire',
 'International Cricket Council (ICC',
 'International Cricket Council',
 'international cricket',
 'the International Cricket Counci

Put the bert part after this 


In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, 
                                  )
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# 2.Document embedding

In [ ]:
document='''Cricket is a bat-and-ball game played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. The batting side scores runs by striking the ball bowled at the wicket with the bat (and running between the wickets), while the bowling and fielding side tries to prevent this (by preventing the ball from leaving the field, and getting the ball to either wicket) and dismiss each batter (so they are "out"). Means of dismissal include being bowled, when the ball hits the stumps and dislodges the bails, and by the fielding side either catching the ball after it is hit by the bat, but before it hits the ground, or hitting a wicket with the ball before a batter can cross the crease in front of the wicket. When ten batters have been dismissed, the innings ends and the teams swap roles. The game is adjudicated by two umpires, aided by a third umpire and match referee in international matches. They communicate with two off-field scorers who record the match's statistical information.
Forms of cricket range from Twenty20, with each team batting for a single innings of 20 overs, to Test matches played over five days. Traditionally cricketers play in all-white kit, but in limited overs cricket they wear club or team colours. In addition to the basic kit, some players wear protective gear to prevent injury caused by the ball, which is a hard, solid spheroid made of compressed leather with a slightly raised sewn seam enclosing a cork core layered with tightly wound string.
The earliest reference to cricket is in South East England in the mid-16th century. It spread globally with the expansion of the British Empire, with the first international matches in the second half of the 19th century. The game's governing body is the International Cricket Council (ICC), which has over 100 members, twelve of which are full members who play Test matches. The game's rules, the Laws of Cricket, are maintained by Marylebone Cricket Club (MCC) in London. The sport is followed primarily in the Indian subcontinent, Australasia, the United Kingdom, southern Africa and the West Indies.[1] Women's cricket, which is organised and played separately, has also achieved international standard. The most successful side playing international cricket is Australia, which has won seven One Day International trophies, including five World Cups, more than any other country and has been the top-rated Test side more than any other country.'''
sentences= document.split('. ')

In [ ]:
#here we tokenize each sentence and convert to ids
#finding maximum length of a sentence (for padding purposes, because bert requires this)
max_len = 0
# For every sentence...
for sentence in sentences:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    sentence_id = tokenizer.encode(sentence, add_special_tokens=True)
    # Update the maximum sentence length.
    max_len = max(max_len, len(sentence_id))
#adding 10 extra to max_len just in case
max_len=max_len+10

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
# input ids is a 2d list, each element is a list(that represents a sentence which is tokenised and converted to the ids)
#attention masks is also a 2d list each element is a list(that represents whether each element of the tokenised list is a padded element or not)
input_ids = []
attention_masks = []
# For every phrase...
for sentence in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the phrase.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the phrase to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      # phrase to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation = True,
                        max_length = 23 ,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    # Add the encoded phrase to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#this is where we call the model and pass the encoded tokens of each sentence
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers.
hidden_states=[]
with torch.no_grad():
    for i in range(len(input_ids)):
      outputs = model(input_ids[i],attention_mask=attention_masks[i])
      hidden_states.append(outputs[2])

print(len(hidden_states[0]))

13


In [ ]:
#here we calculate sentence and document embeddings
# `hidden_states` has shape [no of sentences x 13 x 1 x max_len x 768]
sentence_embeddings=[]
# `token_vecs` is a tensor with shape [22 x 768]
for i in range(len(hidden_states)):
  token_vecs = hidden_states[i][-2][0]#we take the embeddings from the second last layer as it has the best balance in terms of context
  # Calculate the average of all token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  sentence_embeddings.append(sentence_embedding)

sentence_embeddings = torch.stack(sentence_embeddings,dim=0)
document_embedding=torch.mean(sentence_embeddings, dim=0)

# 3.phrase embedding

In [ ]:
#finding maximum length of a phrase (for padding purposes, because bert requires this)
max_len = 0

for phrase in phrases:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    ids = tokenizer.encode(phrase, add_special_tokens=True)
    # Update the maximum phrase length.
    max_len = max(max_len, len(ids))
    
#adding 10 extra to max_len just in case
max_len=max_len+10
print(max_len)

19


In [ ]:
# Tokenize all of the phrases and map the tokens to their word IDs.
input_ids = []
attention_masks = []
# For every phrase...
for phrase in phrases:
    # `encode_plus` will:
    #   (1) Tokenize the phrase.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the phrase to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        phrase,                      # phrase to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation = True,
                        max_length = 23 ,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    # Add the encoded phrase to the list.    
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers.
hidden_states=[]
with torch.no_grad():
    for i in range(len(input_ids)):
      outputs = model(input_ids[i],attention_mask=attention_masks[i])
      hidden_states.append(outputs[2])

In [ ]:
# `hidden_states` has shape [no of sentences x 13 x 1 x max_len x 768]
phrase_embeddings=[]
for i in range(len(hidden_states)):
  token_vecs = hidden_states[i][-2][0]
  # `token_vecs` is a tensor with shape [22 x 768]
  # Calculate the average of all token vectors.
  phrase_embedding = torch.mean(token_vecs, dim=0)
  phrase_embeddings.append(phrase_embedding)

# 4.cosine similarity

In [ ]:
from scipy.spatial.distance import cosine

selected_keyphrases=set()

for i in range(len(phrase_embeddings)):
  diff = 1 - cosine(document_embedding, phrase_embeddings[i])
  print("keyphrase: ",phrases[i])
  print("similarity: ",diff*100,"%\n\n")
  if(diff>=0.675):
    selected_keyphrases.add(phrases[i])

print(selected_keyphrases)

keyphrase:  the United
similarity:  64.50136303901672 %


keyphrase:  the
similarity:  59.27771329879761 %


keyphrase:  the International Cricket Council (
similarity:  70.11339068412781 %


keyphrase:  (
similarity:  59.67443585395813 %


keyphrase:  basic kit
similarity:  66.57353639602661 %


keyphrase:  the International
similarity:  63.796836137771606 %


keyphrase:  international matches in the second half
similarity:  65.09360671043396 %


keyphrase:  19th century
similarity:  60.11441349983215 %


keyphrase:  Empire
similarity:  67.00421571731567 %


keyphrase:  Cricket Council
similarity:  70.85310816764832 %


keyphrase:  Cricket Council (
similarity:  65.77544212341309 %


keyphrase:  Council (ICC
similarity:  55.14177083969116 %


keyphrase:  Marylebone Cricket
similarity:  70.18466591835022 %


keyphrase:  half of the 19th century
similarity:  61.14004850387573 %


keyphrase:  Council (
similarity:  59.6185028553009 %


keyphrase:  Club
similarity:  64.38702344894409 %




In [ ]:
k=1
for i in selected_keyphrases:
  print(k,". ",i,"\n")
  k+=1
print("no of candidate phrases: ",len(phrases))
print("no of selected phrases: ",len(selected_keyphrases))

1 .  the International Cricket 

2 .  ICC 

3 .  referee in international matches 

4 .  the International Cricket Council ( 

5 .  the International Cricket Council 

6 .  International Cricket Council ( 

7 .  Cricket 

8 .  Cricket Council 

9 .  limited overs 

10 .  Marylebone Cricket 

11 .  International Cricket Council (ICC 

12 .  International Cricket Council 

13 .  international cricket 

14 .  first international matches in the second half 

15 .  the International Cricket Council (ICC 

16 .  Marylebone Cricket Club 

17 .  Australia 

18 .  International Cricket 

19 .  Indian subcontinent 

20 .  white kit 

no of candidate phrases:  65
no of selected phrases:  20
